### Plots for logit based

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
from sklearn.metrics import roc_auc_score
import seaborn as sns

df_results = pd.read_csv("./processed_csvs/classification.csv")
df_results = df_results[df_results["dataset"] != "mimic_mortality_prediction"]

In [6]:
df_results["dataset"].unique()

array(['cohort_drug_abuse_classification',
       'cohort_alcohol_abuse_classification',
       'cohort_english_classification',
       'cohort_make_decisions_classification',
       'cohort_abdominal_classification', 'obesity_classification',
       'diabetes_mellitus_classification', 'asthma_classification',
       'cad_classification'], dtype=object)

In [7]:
DATASET_MAP = {
    "asthma_classification": "Obesity Co-Morbidity (Asthma)",
    "cohort_alcohol_abuse_classification": "Cohort Alcohol Abuse",
    "cad_classification": "Obesity Co-Morbidity (CAD)",
    "cohort_drug_abuse_classification": "Cohort Drug Abuse",
    "cohort_english_classification": "Cohort English",
    "cohort_make_decisions_classification": "Cohort Make Decisions",
    "cohort_abdominal_classification": "Cohort Abdominal",
    "diabetes_mellitus_classification": "Obesity Co-Morbidity (Diabetes Mellitus)",
    "obesity_classification": "Obesity",
}

MODEL_MAP = {
    "Llama-2-7b-chat-hf": "Llama-2 (7b)",
    "alpaca-7b": "Alpaca (7b)",
    "asclepius": "Asclepius (7b)",
    "clinical-camel-7b": "Clin-Camel (13b)",
    "medalpaca-7b": "MedAlpaca (7b)",
    "mistral-7b": "Mistral (7b)",
    "Llama-2-13b-chat": "Llama-2 (13b)",
}

DATASET_ORDER_MAP = {
    "asthma_classification": 1,
    "cohort_alcohol_abuse_classification": 6,
    "cad_classification": 2,
    "cohort_drug_abuse_classification": 7,
    "cohort_english_classification": 8,
    "cohort_make_decisions_classification": 9,
    "cohort_abdominal_classification": 5,
    "diabetes_mellitus_classification": 3,
    "obesity_classification": 4,
}

ORDER_MAP = {
    "Llama-2-7b-chat-hf": 2,
    "Llama-2-13b-chat": 1,
    "alpaca-7b": 3,
    "asclepius": 5,
    "clinical-camel-7b": 4,
    "medalpaca-7b": 6,
    "mistral-7b": 0,
}

MODEL_MAP = {
    "Llama-2-7b-chat-hf": "Llama-2 (7b)",
    "alpaca-7b": "Alpaca (7b)",
    "asclepius": "Asclepius (7b)",
    "clinical-camel-7b": "Clin-Camel (13b)",
    "medalpaca-7b": "MedAlpaca (7b)",
    "mistral-7b": "Mistral (7b)",
    "Llama-2-13b-chat": "Llama-2 (13b)",
}

colors = sns.color_palette("deep", 10, desat=0.8)
COLOR_MAP = {
    "Mistral (7b)": colors[7],
    "Llama-2 (7b)": colors[1],
    "Llama-2 (13b)": colors[0],
    "Alpaca (7b)": colors[3],
    "Asclepius (7b)": colors[1],
    "Clin-Camel (13b)": colors[0],
    "MedAlpaca (7b)": colors[3],
}

COLOR_MAP_LOWER = {
    "mistral-7b": colors[7],
    "Llama-2-7b-chat-hf": colors[1],
    "Llama-2-13b-chat": colors[0],
    "alpaca-7b": colors[3],
    "asclepius": colors[1],
    "clinical-camel-7b": colors[0],
    "medalpaca-7b": colors[3],
}

In [8]:
def convert_to_float_list(string_list):
    return [float(x) for x in string_list.strip("[]").split(", ")]


# Apply the function to the 'probabilities' column
df_results["probabilities"] = df_results["probabilities"].apply(convert_to_float_list)
df_results["positive_probabilities"] = [p[1] for p in df_results["probabilities"]]

# Compute AUROC

In [7]:
all_datasets = df_results["dataset"].unique()
all_models = df_results["model"].unique()
all_annotators = df_results["annotator"].unique()
df_results = df_results.sort_values(by=["dataset", "model", "annotator"])
all_auroc = []

df_list = []

for dataset in all_datasets:
    for model in all_models:
        for annotator in all_annotators:
            df = df_results[
                (df_results["dataset"] == dataset)
                & (df_results["model"] == model)
                & (df_results["annotator"] == annotator)
            ].copy()
            labels = df["gold_class"].unique()
            auroc = roc_auc_score(df["gold_class"], df["positive_probabilities"])
            df["auroc"] = auroc
            df_list.append(df)

df_results = pd.concat(df_list)

In [8]:
df_results = df_results.drop(["instance", "probabilities"], axis=1)

df_results_grouped = (
    df_results.groupby(["task_type", "dataset", "model", "annotator"])
    .mean()
    .reset_index()
)

In [9]:
df_results_grouped["model"].unique()

array(['Llama-2-13b-chat', 'Llama-2-7b-chat-hf', 'alpaca-7b', 'asclepius',
       'clinical-camel-7b', 'medalpaca-7b', 'mistral-7b'], dtype=object)

In [10]:
df_results_grouped = df_results_grouped.rename(columns={"auroc": "AUROC"})

# drop any rows containing -1

df_results_grouped = df_results_grouped[(df_results_grouped["AUROC"] != -1)]

df_results_grouped["y_labels"] = df_results_grouped["model"].apply(
    lambda x: MODEL_MAP[x]
)

In [11]:
DATASET_ORDER_MAP = {
    "asthma_classification": 1,
    "cohort_alcohol_abuse_classification": 6,
    "cad_classification": 2,
    "cohort_drug_abuse_classification": 7,
    "cohort_english_classification": 8,
    "cohort_make_decisions_classification": 9,
    "cohort_abdominal_classification": 5,
    "diabetes_mellitus_classification": 3,
    "obesity_classification": 4,
}

ORDER_MAP = {
    "Llama-2-7b-chat-hf": 2,
    "Llama-2-13b-chat": 1,
    "alpaca-7b": 3,
    "asclepius": 5,
    "clinical-camel-7b": 4,
    "medalpaca-7b": 6,
    "mistral-7b": 0,
}

df_results_grouped["order"] = df_results_grouped["model"].apply(lambda x: ORDER_MAP[x])
df_results_grouped["order2"] = df_results_grouped["dataset"].apply(
    lambda x: DATASET_ORDER_MAP[x]
)

df_results_grouped = df_results_grouped.sort_values(by=["order"])

In [12]:
datasets = [
    "asthma_classification",
    "cad_classification",
    "obesity_classification",
    "diabetes_mellitus_classification",
    "cohort_abdominal_classification",
    "cohort_alcohol_abuse_classification",
    "cohort_drug_abuse_classification",
    "cohort_english_classification",
    "cohort_make_decisions_classification",
]

In [16]:
fig, ax = plt.subplots(
    nrows=3,
    ncols=3,
    figsize=(34, 34),
    dpi=300,
    sharex=True,
    sharey=True,
    layout="constrained",
)
sns.set_style("white")
sns.set_context("poster")
sns.set_palette("deep", n_colors=12)
sns.set_theme(style="white", font_scale=3)
sns.set_context("paper", font_scale=3)

for idx, dataset in enumerate(datasets):

    df_results_dataset = df_results_grouped[(df_results_grouped["dataset"] == dataset)]
    models = df_results_grouped["y_labels"]
    annotators = df_results_grouped["annotator"]
    sns.set_theme(style="white", font_scale=3)

    # ax.set_ylim(0, 1.02)
    # plot points

    palette = sns.color_palette("crest")
    sns.set_theme(style="white", font_scale=1.5)
    mon = sns.violinplot(
        data=df_results_dataset,
        x="AUROC",
        y="y_labels",
        hue="model",
        palette=COLOR_MAP_LOWER,
        cut=0.0,
        legend=False,
        ax=ax[idx // 3][idx % 3],
    )
    mon.set_title(DATASET_MAP[dataset], fontsize=35, pad=30)
    mon.axhline(3.5, color="black", linestyle="--", linewidth=1.25)
    if idx == 2:
        mon.text(
            0.92,
            3.85,
            "Clinical models",
            ha="center",
            va="center",
            style="italic",
            fontsize=28,
        )
        mon.text(
            0.92,
            3.15,
            "General models",
            ha="center",
            va="center",
            style="italic",
            fontsize=28,
        )
    mon.set(ylabel=None)
    mon.set(xlabel=None)
    plt.xlim(0, 1.05)
    plt.tight_layout()
    sns.despine(top=True, right=True, left=False, bottom=False)
    sns.set_theme(style="white", font_scale=3)

    # do this with idx
    # if (idx // 3) == 0 and (idx % 3) == 0:
    #     sns.despine(top=True, right=True, left=False, bottom=True)
    # if (idx // 3) == 0 and (idx % 3) == 1:
    #     sns.despine(top=True, right=True, left=True, bottom=True)
    # if (idx // 3) == 0 and (idx % 3) == 2:
    #     sns.despine(top=True, right=True, left=True, bottom=True)
    # if (idx // 3) == 1 and  (idx % 3) == 0:
    #     sns.despine(top=True, right=True, left=False, bottom=True)
    # if (idx // 3) == 1 and  (idx % 3) == 1:
    #     sns.despine(top=True, right=True, left=True, bottom=True)
    # if (idx // 3) == 1 and (idx % 3) == 2:
    #     sns.despine(top=True, right=True, left=True, bottom=True)
    # if (idx // 3) == 2 and (idx % 3) == 0:
    #     sns.despine(top=True, right=True, left=False, bottom=False)
    # if (idx // 3) == 2 and (idx % 3) == 1:
    #     sns.despine(top=True, right=True, left=True, bottom=False)
    # if (idx // 3) == 2 and (idx % 3) == 2:
    #     sns.despine(top=True, right=True, left=True, bottom=False)


fig.text(0.5, 0, "AUROC", ha="center")

plt.tight_layout()
plt.savefig("classification.pdf")

/tmp/ipykernel_90325/2250890429.py:62: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
